# Hiati

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import path_results

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )

In [ ]:
plt.plot(gmst_ctrl.GMST)
plt.plot(gmst_rcp.GMST )


In [ ]:
np.polyfit(np.arange(15), gmst_ctrl.GMST.rolling({'year':15}),1)